# Content Based Recommendation

In [38]:
import os
import pandas as pd
import numpy as np

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords


## Data Preprocessing

In [54]:
DATA_DIR = os.path.abspath('./datasets/output/data_2.csv')
df = pd.read_csv(DATA_DIR,index_col=0)
df.head()


,class,text
id,,
182,Interview,"﻿विडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फ..."
127,Interview,﻿नेपाली कांग्रेसको सभापतिमा रामचन्द्र पौडेलको ...
85,Interview,﻿\nदुर्भाग्य नै भन्नुपर्छ हाम्रो प्राथमिकता पे...
162,Interview,"﻿नयनराज पाण्डे, लेखक\nविमोचन हुनै लागेको तपाईं..."
13,Interview,"﻿- भवन भट्ट, उपाध्यक्ष, गैर आवासीय नेपाली संघ\..."


In [55]:
df['id'] = df.index
df

,class,text,id
id,,,
182,Interview,"﻿विडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फ...",182
127,Interview,﻿नेपाली कांग्रेसको सभापतिमा रामचन्द्र पौडेलको ...,127
85,Interview,﻿\nदुर्भाग्य नै भन्नुपर्छ हाम्रो प्राथमिकता पे...,85
162,Interview,"﻿नयनराज पाण्डे, लेखक\nविमोचन हुनै लागेको तपाईं...",162
13,Interview,"﻿- भवन भट्ट, उपाध्यक्ष, गैर आवासीय नेपाली संघ\...",13
...,...,...,...
68,Entertainment,﻿उनले ‘टिमवर्क’ र ‘लिडरसिप’ फिल्ममेकिङको महत्व...,68
605,Entertainment,रुपा बनिन् हेरिटेज क्वीन नेपाल\n\n\nराजधानीको ...,605
294,Entertainment,﻿प्रकाश र किरणको अर्थ एउटै भए पनि कालो पोथी का...,294


In [56]:
df.dropna(inplace=True) # drop rows with invalid values

## Processing data

In [57]:
nepali_stop_words = set(stopwords.words('nepali'))

tf = TfidfVectorizer(stop_words=nepali_stop_words,encoding='utf-8',decode_error='ignore')
tf = TfidfVectorizer(tokenizer= lambda x: x.split(" "),
                                  sublinear_tf=True, encoding='utf-8',
                                  decode_error='ignore',
                                  max_df=0.5,
                                  min_df=10,
                                  stop_words=nepali_stop_words)

tfidf_matrix = tf.fit_transform(df['text'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

c:\users\dell\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['कम', 'से'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [61]:
for idx, row in df.iterrows():
    try:
        idx = int(idx)
        similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
        similar_items = [(cosine_similarities[idx][i], df['id'][i], df['class'][i]) for i in similar_indices]

        results[row['id']] = similar_items[1:]
    except: pass
    
print('Completed!')
similar_indices = cosine_similarities[idx].argsort()[:-100:-1]

Completed!


## Displaying Results

In [62]:
def printBold(string):
    display(Markdown(string))

def item(id):
    id = str(id)
    return {'text': df.loc[df['id'] == id]['text'].tolist()[0].split(' - ')[0],
           'class': df.loc[df['id'] == id]['class'].tolist()[0].split(' - ')[0]
           }

def recommender(item_id, num): 
    print("\n")
    printBold("**Recommendation System**")
    print("The given news " + "'"+ item(item_id)['text'][:100] + "..." + "'" + " is of Category: " + str(item(item_id)['class'][:100]))
    
    print("\nThe " + str(num) + " most similar news in ascending order is given below: ")
    
    recs = results[str(item_id)][:num]
    for index, rec in enumerate(recs):
        print("\n")
        print(str(index+1) + ") " + "Category: " + str(rec[2]) + "\n\n" + "News: " + "'" + item(rec[1])['text'][:200] + "..." + "'" +" (score:" + str(rec[0]) + ")")

# Extract the news (that the user is currently interacting with) and provide 8 recommended news
recommender(item_id=120, num=10)

**Recommendation System**

The given news '﻿
वसन्त अधिकारी निर्देशक/प्रवक्ता, यातायात व्यवस्था विभाग
सार्वजनिक यातायात सुधार्न के गर्नुपर्छ ?
न...' is of Category: Interview

The 10 most similar news in ascending order is given below: 


1) Category: Interview

News: '﻿
व्यवस्थापिका संसद्को अर्थ समितिका सभापति हुन्, सांसद प्रकाश ज्वाला । नेपाल स्रोत भएर पनि परिचालन गर्न नसक्दा विकासमा पछि परेको उनको बुझाइ छ । अहिले चर्चामा रहेको ‘फास्ट ट्रयाक’ भारतीय कम्पनीलाई दिन ...' (score:0.20815232111931387)


2) Category: Business

News: '﻿
﻿सूचना प्रविधिका क्षेत्रमा २१ वर्षे लामो अनुभव संगालेका इन्जिनियर विनोद ढकाल कम्प्युटर एसोसिएसन नेपाल महासंघका अध्यक्ष छन् । व्यवस्थापनमा स्नातकोत्तर र भारतको मदरासको द इन्स्टिच्युसन अफ इन्जिनियरिङब...' (score:0.20815232111931387)


3) Category: Business

News: '﻿आन्दोलनरत मधेशी दलहरूलाई सहमतिमा ल्याउन सरकारले संसद्बाट गराएको संविधान संशोधन नेपालका लागि दीर्घकालीन रुपमा खुट्टामा बञ्चरो हाने समान हुनेछ ।
संविधानमा भर्खरै भएको संशोधनले भविष्यमा आफ्नो राजनीतिक प...' (score:0.18829638843999

In [63]:
import pickle
with open('similarity_matrix.pkl','wb') as f:
    pickle.dump(results,f)